In [1]:
cd /content/drive/My Drive/CoQUY/Data_coQuy/Kho ngu lieu 10000 cau duoc gan nhan tu loai

/content/drive/My Drive/CoQUY/Data_coQuy/Kho ngu lieu 10000 cau duoc gan nhan tu loai


In [2]:
import os 
import pandas as pd
import joblib
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from gensim.models import KeyedVectors
import numpy as np

In [3]:
raw_data=[]
for i in os.listdir("/content/drive/My Drive/CoQUY/Data_coQuy/Kho ngu lieu 10000 cau duoc gan nhan tu loai"):  
        with open(i) as f:
          raw_data.append(f.read())

In [5]:
data_X=[]
for i in range(len(raw_data)):
  b=raw_data[i].split('\n')
  data_X.append(b)

In [6]:
X=[]
for i in data_X:
    for j in i:
        X.append(j)

# 10 521 sentences 

In [8]:
len(X)

10521

In [9]:
X_test, X_train  = train_test_split(X, test_size=0.7,random_state=101)
X_test, X_val  = train_test_split(X_test, test_size=0.5,random_state=101)
print(len(X_train),len(X_test),len(X_val))

# Save train : train.csv ( 70%)

In [11]:
data_train=[]
for senten in X_train:
    senten_split=senten.split()
    for wp in senten_split:
        data_train.append(wp)
words_train=[]
pos_tag_train=[]
for i in data_train:
    if (i.count("/")==1):
        index=i.split("/")
        if( index[-1].isalpha() or index[-1]=="(" or index[-1]==")" or index[-1]==":" or index[-1]=="." or index[-1]=="," or index[-1]== ";" or index[-1]=="?"  or index[-1]=="!" or index[-1]=="+" or index[-1]=="-" or index[-1]=="..." or index[-1]=="<" or index[-1]==">" or index[-1]=="{" or index[-1]=="}"):
            words_train.append(index[0]) 
            pos_tag_train.append(index[-1])
df_train=pd.DataFrame(columns=("words","pos_tag"))
df_train["words"]=words_train
df_train["pos_tag"]=pos_tag_train
df_train.to_csv('/content/drive/MyDrive/CoQUY/train.csv')

# Save validation:  val.csv (15%)

In [12]:
data_val=[]
for senten in X_val:
    senten_split=senten.split()
    for wp in senten_split:
        data_val.append(wp)
words_val=[]
pos_tag_val=[]
for i in data_val:
    if (i.count("/")==1):
        index=i.split("/")
        if( index[-1].isalpha() or index[-1]=="(" or index[-1]==")" or index[-1]==":" or index[-1]=="." or index[-1]=="," or index[-1]== ";" or index[-1]=="?"  or index[-1]=="!" or index[-1]=="+" or index[-1]=="-" or index[-1]=="..." or index[-1]=="<" or index[-1]==">" or index[-1]=="{" or index[-1]=="}"):
            words_val.append(index[0]) # ( ) : . , ; ? ! + - ... < > { }
            pos_tag_val.append(index[-1]) # ( ) : . , ; ? ! +
df_val=pd.DataFrame(columns=("words","pos_tag"))
df_val["words"]=words_val
df_val["pos_tag"]=pos_tag_val
df_val.to_csv('/content/drive/MyDrive/CoQUY/val.csv')

# Save test : test.csv (  15%)

In [13]:
data_test=[]
for senten in X_test:
    senten_split=senten.split()
    for wp in senten_split:
        data_test.append(wp)
words_test=[]
pos_tag_test=[]
for i in data_test:
    if (i.count("/")==1):
        index=i.split("/")
        if( index[-1].isalpha() or index[-1]=="(" or index[-1]==")" or index[-1]==":" or index[-1]=="." or index[-1]=="," or index[-1]== ";" or index[-1]=="?"  or index[-1]=="!" or index[-1]=="+" or index[-1]=="-" or index[-1]=="..." or index[-1]=="<" or index[-1]==">" or index[-1]=="{" or index[-1]=="}"):
            words_test.append(index[0])
            pos_tag_test.append(index[-1])
df_test=pd.DataFrame(columns=("words","pos_tag"))
df_test["words"]=words_test
df_test["pos_tag"]=pos_tag_test
df_test.to_csv('/content/drive/MyDrive/CoQUY/test.csv')

# Load Word Embedding

In [14]:
word_emb = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/CoQUY/baomoi.model.bin", binary=True)

# Feature Extraction

In [15]:
X_features_train=[]
Y_labels_train=[]
def convert(x):
    return dictionary[x]

set_pos_train=set (pos_tag_train)
dictionary = {}

for idx, i in enumerate(set_pos_train):
    dictionary[i]=idx

#df_train["pos_tag"]=df_train["pos_tag"].apply(lambda x: convert(x))

for i in pos_tag_train:
    Y_labels_train.append(i)

for idx,w in enumerate(df_train["words"]):
    if (w in word_emb.vocab.keys()):
        X_features_train.append(word_emb.word_vec(w))
    else:
        X_features_train.append(np.zeros(400,dtype=float))

In [16]:
X_features_val=[]
Y_labels_val=[]
def convert(x):
    return dictionary[x]

set_pos_val=set (pos_tag_val)
dictionary = {}

for idx, i in enumerate(set_pos_val):
    dictionary[i]=idx

#df_val["pos_tag"]=df_val["pos_tag"].apply(lambda x: convert(x))

for i in pos_tag_val :
    Y_labels_val.append(i)

for idx,w in enumerate(df_val["words"]):
    if (w in word_emb.vocab.keys()):
        X_features_val.append(word_emb.word_vec(w))
    else:
        X_features_val.append(np.zeros(400,dtype=float))

In [17]:
X_features_test=[]
Y_labels_test=[]
def convert(x):
    return dictionary[x]

set_pos_test=set (pos_tag_test)
dictionary = {}

for idx, i in enumerate(set_pos_test):
    dictionary[i]=idx


for i in pos_tag_test :
    Y_labels_test.append(i)

for idx,w in enumerate(df_test["words"]):
    if (w in word_emb.vocab.keys()):
        X_features_test.append(word_emb.word_vec(w))
    else:
        X_features_test.append(np.zeros(400,dtype=float))

In [18]:
print(len(X_features_train),len(Y_labels_train))
print(len(X_features_val),len(Y_labels_val))
print(len(X_features_test),len(Y_labels_test))

149946 149946
33167 33167
32578 32578


In [ ]:
#model1=SVC().fit(X_features_train,Y_labels_train)
#predict=model1.predict(X_features_val)

In [ ]:
#predict=model1.predict(X_features_val)
print('Accuracy: ',accuracy_score(Y_labels_val, predict))
print('Classification report: \n',classification_report(Y_labels_val, predict))
print('Confusion matrix: \n',confusion_matrix(Y_labels_val, predict))

Accuracy:  0.7358519009859198


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification report: 
               precision    recall  f1-score   support

           !       0.00      0.00      0.00        82
           (       0.00      0.00      0.00       124
           )       0.00      0.00      0.00       128
           +       0.00      0.00      0.00         1
           ,       0.23      1.00      0.38      1832
           -       0.00      0.00      0.00       247
           .       0.00      0.00      0.00      1241
         ...       0.00      0.00      0.00       231
           :       1.00      1.00      1.00       241
           ;       1.00      1.00      1.00        19
           >       1.00      1.00      1.00         8
           ?       0.00      0.00      0.00        60
           A       0.89      0.78      0.83      2030
          Ab       0.00      0.00      0.00         2
           B       0.00      0.00      0.00         4
           C       0.91      0.80      0.86      1245
           E       0.79      0.91      0.84      2110
  

In [ ]:
#filename = '/content/drive/MyDrive/CoQUY/model_SVM_result.sav' 
#joblib.dump(model1, filename)

['/content/drive/MyDrive/CoQUY/model_SVM_result.sav']

In [ ]:
#result_test=model_loaded_result.predict(X_features_test)
print('Accuracy: ',accuracy_score(Y_labels_test, result_test))
print('Classification report: \n',classification_report(Y_labels_test, result_test))
print('Confusion matrix: \n',confusion_matrix(Y_labels_test, result_test))

Accuracy:  0.7366320830007981


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Classification report: 
               precision    recall  f1-score   support

           !       0.00      0.00      0.00       104
           (       0.00      0.00      0.00        94
           )       0.00      0.00      0.00        93
           ,       0.24      1.00      0.39      1865
           -       0.00      0.00      0.00       265
           .       0.00      0.00      0.00      1263
         ...       0.00      0.00      0.00       208
           :       1.00      1.00      1.00       236
           ;       1.00      1.00      1.00        12
           >       1.00      1.00      1.00         2
           ?       0.00      0.00      0.00        65
           A       0.88      0.79      0.83      2033
          Ab       0.00      0.00      0.00         2
           B       0.00      0.00      0.00         4
           C       0.91      0.79      0.84      1153
           E       0.79      0.92      0.85      2018
           I       1.00      0.15      0.26        20
  

# Word segmentation: pyvi

In [19]:
! pip install pyvi
from pyvi import ViTokenizer

     |████████████████████████████████| 8.5MB 3.3MB/s 
     |████████████████████████████████| 747kB 54.0MB/s 


# **DEMO**

In [20]:
model_loaded_result=joblib.load("/content/drive/MyDrive/CoQUY/model_SVM_result.sav")
word_emb = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/CoQUY/baomoi.model.bin", binary=True)
def raw_senten(sentence):
    list_senten=[]
    np_senten=[]
    list_senten=(ViTokenizer.tokenize(sentence)).split()
    return list_senten

def raw_word(list_senten):
    np_=[]
    for i in list_senten:
        if (i in word_emb.vocab.keys()):
            np_.append((word_emb.word_vec(i)).reshape(1,400))
        else:
            np_.append((np.zeros(400,dtype=float)).reshape(1,400))
    return np_

def demo(sentence):
    list_senten=raw_senten(sentence)
    np_=raw_word(list_senten)
    a=[]
    for i in np_:
        a.append(model_loaded_result.predict(i))
    annotated_tool=''
    for i in range (len(list_senten)):
        #annotated_tool=''
        annotated_tool=annotated_tool+(list_senten[i])+"/"+a[i][0]+" "
    return annotated_tool



# ANNOTATION TOOL

In [28]:
sentence=str(input())
print(demo(sentence))

hôm nay trời mưa rất to.
hôm_nay/N trời/N mưa/V rất/R to/A ./, 


# Demo sentences in file.txt

In [ ]:
with open("/content/drive/MyDrive/CoQUY/Train_anotator/50 CÂU TUOITRE/DATA.txt",'r') as f:
    data=f.read()   
data=data.split("\n")
data_anno=""    
for i in data:
    data_annotated=demo(i)
    data_anno=data_anno+data_annotated+"\n"
data_anno=data_anno[:-2]
with open("/content/drive/MyDrive/CoQUY/Train_anotator/data_annotated.txt","w") as f1:
    f1.write(data_anno)